In [1]:
import os
from binance.client import Client
from binance.enums import *
import pandas as pd
from time import sleep


api_key = 'api key'
api_secret = 'secret key'


client = Client(api_key, api_secret)


In [2]:
# DCA parameters
symbol = 'BTCUSDT'  
initial_investment_amount = 100  
interval = 60 * 60 * 24  
total_investment = 1500  
price_drop_threshold = 0.08  
profit_threshold = 0.05  
leverage = 10 


In [3]:
def get_current_price(symbol):
    avg_price = client.futures_mark_price(symbol=symbol)
    return float(avg_price['markPrice'])

def place_market_order(symbol, quantity, side):
    try:
        order = client.futures_create_order(
            symbol=symbol,
            side=side,
            type=ORDER_TYPE_MARKET,
            quantity=quantity
        )
        return order
    except Exception as e:
        print(f"An exception occurred: {e}")
        return None

def calculate_quantity(investment_amount, current_price, leverage):
    return round((investment_amount * leverage) / current_price, 6)

def dca_bot(symbol, initial_investment_amount, interval, total_investment, price_drop_threshold, profit_threshold, leverage):
    invested_amount = 0
    investment_amount = initial_investment_amount
    initial_price = get_current_price(symbol)
    last_price = initial_price
    total_quantity = 0

    while invested_amount < total_investment:
        current_price = get_current_price(symbol)

    
        if current_price <= last_price * (1 - price_drop_threshold):
            investment_amount *= 2
            last_price = current_price 

        quantity = calculate_quantity(investment_amount, current_price, leverage)
        print(f"Placing order for {quantity} {symbol} at price {current_price}")
        order = place_market_order(symbol, quantity, SIDE_BUY)
        
        if order:
            invested_amount += investment_amount
            total_quantity += quantity
            print(f"Total invested: {invested_amount} USD")
            print(f"Total quantity: {total_quantity}")
        else:
            print("Order failed, retrying after interval")
        
        sleep(interval)
  
        current_value = total_quantity * current_price / leverage
        if current_value >= invested_amount * (1 + profit_threshold):
            print(f"Profit threshold reached: current value = {current_value}, invested amount = {invested_amount}")
            sell_order = place_market_order(symbol, total_quantity, SIDE_SELL)
            if sell_order:
                print(f"Sold {total_quantity} {symbol} at price {current_price}")
                break
            else:
                print("Sell order failed, retrying after interval")

    print("DCA completed")


In [ ]:
dca_bot(symbol, initial_investment_amount, interval, total_investment, price_drop_threshold, profit_threshold, leverage)